In [1]:
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen
import file_load
import librosa
import numpy as np
import csv

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [2]:
def wavenet_encode(wave):
    model_path = './wavenet-ckpt/wavenet-ckpt/model.ckpt-200000' #模型位置
    # audio = np.load(file_path)
    encoding = fastgen.encode(wave, model_path, len(wave))
    print(encoding.reshape((-1, 16)).shape)
    return encoding.reshape((-1, 16))

In [3]:
dataset_path = './survry20tracks'#dataset位置

name1 = '.mp3' #欲讀取的檔名
name2 = '.npy' 

mp3_paths, mp3_names, folder_list, folder_name = file_load.file_path(dataset_path, name1)
npy_paths, npy_names, folder_list, folder_name = file_load.file_path(dataset_path, name2)
replace_paths = []
replace_names = []

for npy_path, npy_name in zip(npy_paths, npy_names):
    temp = npy_path.replace('.npy', '.mp3')
    temp2 = npy_name.replace('.npy', 'mp3')
    replace_paths.append(temp)
    replace_names.append(temp2)

comparison_paths = list(set(mp3_paths).difference(set(replace_paths)))
comparison_names = list(set(mp3_names).difference(set(replace_names)))
print('資料筆數：',len(comparison_names))

#print('資料名稱：',mp3_names)

for count in range (0,len(comparison_names)):
    print(count,'|',mp3_names[count])

資料筆數： 20
0 | 1au9q3wiWxIwXTazIjHdfF
1 | 1ExfPZEiahqhLyajhybFeS
2 | 1fLlRApgzxWweF1JTf8yM5
3 | 1pSIQWMFbkJ5XvwgzKfeBv
4 | 1UMJ5XcJPmH6ZbIRsCLY5F
5 | 2W2eaLVKv9NObcLXlYRZZo
6 | 3S0OXQeoh0w6AY8WQVckRW
7 | 3wF0zyjQ6FKLK4vFxcMojP
8 | 4FCb4CUbFCMNRkI6lYc1zI
9 | 4RL77hMWUq35NYnPLXBpih
10 | 52UWtKlYjZO3dHoRlWuz9S
11 | 5b88tNINg4Q4nrRbrCXUmg
12 | 5E5MqaS6eOsbaJibl3YeMZ
13 | 5uCax9HTNlzGybIStD3vDh
14 | 5WLSak7DN3LY1K71oWYuoN
15 | 6G7URf5rGe6MvNoiTtNEP7
16 | 6QPKYGnAW9QozVz2dSWqRg
17 | 6rUp7v3l8yC4TKxAAR5Bmx
18 | 7qjbpdk0IYijcSuSYWlXO6
19 | 7uRznL3LcuazKpwCTpDltz


In [4]:
def wavenet_vector(dataset_path):
    name1 = '.mp3' #欲讀取的檔名
    name2 = '.npy' 

    mp3_paths, mp3_names, folder_list, folder_name = file_load.file_path(dataset_path, name1)
    npy_paths, npy_names, folder_list, folder_name = file_load.file_path(dataset_path, name2)

    replace_paths = []
    replace_names = []

    for npy_path, npy_name in zip(npy_paths, npy_names):
        temp = npy_path.replace('.npy', '.mp3')
        temp2 = npy_name.replace('.npy', 'mp3')
        replace_paths.append(temp)
        replace_names.append(temp2)

    comparison_paths = list(set(mp3_paths).difference(set(replace_paths)))
    comparison_names = list(set(mp3_names).difference(set(replace_names)))

    print('沒跑過的數量為{}'.format(str(len(comparison_names))))
    
    with open('survry20tracks/tracks_survey20tracks_vector.csv', 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)
        #writer.writerow(['name','audio_vector'])                         
        count = 0
        for mp3, name in zip(comparison_paths, comparison_names):
            # print(mp3)
            wave, sr = librosa.load(mp3, sr = 16000)
            wavenet_data = wavenet_encode(wave)
            std_wavenet = np.std(wavenet_data, axis=0)
            mean_wavenet = np.mean(wavenet_data, axis=0)

            average_difference_channels = np.zeros((16,))

            for i in range(0, len(wavenet_data) - 2, 2):
                temp = wavenet_data[i] - wavenet_data[i+1]
                average_difference_channels += temp
            average_difference_channels /= (len(wavenet_data) // 2)   
            average_difference_channels = np.array(average_difference_channels)

            concat_features_wavenet = np.hstack((std_wavenet, mean_wavenet))
            concat_features_wavenet = np.hstack((concat_features_wavenet, average_difference_channels))
            
            save_path = mp3.replace('.mp3', "")
            string = save_path
            num = string[-5:]
            print('歌曲編號：',num)
                
            #print(concat_features_wavenet)
            print('save',concat_features_wavenet.shape,'in',save_path)
            print('===================================================================')
            np.save(save_path,concat_features_wavenet) #存特徵向量在原資料夾
               
            # 寫入CSV
            # Appending new rows to csv file 'a'        
            Table = [[num,concat_features_wavenet]] 
            writer.writerows(Table)
            count += 1
            print('剩餘數量：',len(comparison_names)-count)
    
    print("===== Write CSV DONE =====")

In [5]:
if __name__ == "__main__":
    dataset_path = './survry20tracks'#dataset位置
    wavenet_vector(dataset_path)
    print("========== FINISH ==========")

沒跑過的數量為20
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
歌曲編號： WlXO6
save (48,) in ./survry20tracks\7qjbpdk0IYijcSuSYWlXO6
剩餘數量： 19
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
歌曲編號： KfeBv
save (48,) in ./survry20tracks\1pSIQWMFbkJ5XvwgzKfeBv
剩餘數量： 18
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
歌曲編號： D3vDh
save (48,) in ./survry20tracks\5uCax9HTNlzGybIStD3vDh
剩餘數量： 17
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
歌曲編號： VckRW
save (48,) in ./survry20tracks\3S0OXQeoh0w6AY8WQVckRW
剩餘數量： 16
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
歌曲編號： SWqRg
save (48,) in ./survry20track

In [82]:
# 排序歌曲name
import operator
data = csv.reader(open('tracks_audiofeatures0625_re/audio_vector.csv'))
sortedlist = sorted(data, key=operator.itemgetter(0)) # 0 specifies according to first column we want to sort
with open("tracks_audiofeatures0625_re/audio_vector_sort.csv", "w") as f:
    fileWriter = csv.writer(f)
    fileWriter.writerow(['num','audio_vector'])
    for row in sortedlist:
        fileWriter.writerow(row)
    print('===== Sort Done =====')

===== Sort Done =====


In [83]:
# 排序歌曲name
import operator
data = csv.reader(open('tracks_audiofeatures0625_re/audio_vector.csv'))
sortedlist = sorted(data, key=operator.itemgetter(0)) # 0 specifies according to first column we want to sort
with open("tracks_audiofeatures0625_re/audio_vector_sort_new.csv", "w", newline='') as f:
    fileWriter = csv.writer(f)
    fileWriter.writerow(['num','audio_vector'])
    for row in sortedlist:
        fileWriter.writerow(row)
    print('===== Sort Done =====')

===== Sort Done =====


In [91]:
'''
import csv
with open("tracks_audiofeatures0622_re/audio_vector.csv","r") as f:
    csvreader = csv.reader(f)
    for row in csvreader:
        #print(row[0])
        if "name" in row[0]:
            print("OK")
'''

OK
